---
layout: post
title: Terraform Automation Plan
description: A guide to AWS automation using Terraform.  The project and purpose of this automation is to provide Student Desktops in the cloud.
author: Aaron Anand, Aadit Gupta
courses: { csp: {week: 20, categories: [6.B]}, csa: {week: 20} }
categories: [C7.0, C7.1, C7.2]
type: devops
---

## Automation Planning
`Cloud Computing`.  Traditionally, classroom have been service by on premise  desktops; costs can be high and aging is rapid. The purpose of this project is to build workspaces in the cloud; students would use their District issued Chromebook to access these workspaces.   Numbers say this could be much more affordable.  However, a key concern in this process is to build, scale, and manager these workpaces.

Automating Cloud Setups.  The following illustration shows a `Terraform` recipe being pushed to the `AWS EC2` computer and generating an `Ubuntu Linux` workspace.   The names `kasm1, kasm2, ...` represent machines using `KASM workspaces`, a containerized desktop software provider.  

```
            AWS EC2 Ubuntu Linux instances scaled (1 to N)
            |-> kasm1.ncs.com - 5 users logins / 3 active sessions
            |
            |-> kasm2.ncs.com - 5 users logins / 3 active sessions
Terraform-->|
 (recipe)   |-> kasm3.ncs.com - 5 users logins / 3 active sessions
            |
            |-> kasm4.ncs.com - 5 users logins / 3 active sessions
            |
            |-> kasmN.ncs.com - 5 users logins / 3 active sessions

```

`Scaling`. To scale numbers, each machine can have up to `5 user logins` per instance, but according to resources would only adequetly serve `3 active session`.  There are many additional Terraform, AWS and/or KASM features to allocate resources, scal resources, and timeout inactive workspace sessions.  Additionally, `machines can be put to sleep`, for instance 11pm to 7am daily to save on cost.  Or, `woken up` based off of demand weekends and holidays.


### Setup Requirements (Part 1 - System)
`System`. These are features that get the KASM system running on AWS EC2 Ubuntu.    The specification have been captured by proof of concept analysis.

#### AWS EC2 Instances
Configuration
- Region Oregon (us-west-2)
- Instance t2.medium (2 x CPU, 4 Gib RAM)
- Architecture x86
- Use “kasm” key pair
- Storage 60 Gib gp3 general purpose SD
- Allow SSH, HTTP, HTTPS

#### KASM single server install 1.13.1 
Shell commands

```bash
# Reference: https://kasmweb.com/docs/latest/install/single_server_install.html
cd /tmp
curl -O https://kasm-static-content.s3.amazonaws.com/kasm_release_1.13.1.421524.tar.gz
tar -xf kasm_release_1.13.1.421524.tar.gz

# Install prompts for Swap Partition, use 8 Gib
sudo bash kasm_release/install.sh
```

### Setup Requirements (Part 2 - Networking)
`Networking`.  The purpose of this step is to provide friendly internet access names to the students.  Additionally, we hope to enable as much setup and verification of these steps in Terraform and AWS CLI as possible.
- AWS EC2 elastic IPs, permanent ips
- AWS Route 53, kasm DNS names to elastic IPs
- Nginx configuration to reverse proxy kasm DNS name to port
- Certbot certificates for DNS name to HTTPs

### Setup Requirements (Part 3 - Workspaces)
`Workspace`.  The purpose of this step is to configure KASM workspaces to meet the needs of CSSE, CSP, and CSA classes.  This includes all development tools, these described by Shell and Docker scripts that follow.  Common developer tools like `make` were found missing in KASM, since KASM is built using Docker `sudo apt` will not be allowed, so tools need to be comprehensive.

Docker Workspace and Registry must match version requirements of system setup (1.13.1)
    - Docker wks Video https://www.youtube.com/watch?v=BGP69_f1wq0
    - Registry Video https://www.youtube.com/watch?v=Y6ggn9McIwI

In [ ]:
#!/bin/bash

# SYSTEM TOOLS describe by make setup

# Function to check if a line exists in the .bashrc file
line_exists_in_bashrc() {
  grep -Fxq “$1" ~/.bashrc
}
# Function to add lines to the .bashrc file if they don’t exist
add_to_bashrc() {
  if ! line_exists_in_bashrc “$1”; then
    echo “$1" >> ~/.bashrc
  fi
}
# Function to install Visual Studio Code
install_vscode() {
  if ! command -v code &>/dev/null; then
    echo “Installing Visual Studio Code...”
    # Add VSCode repository and key
    wget -qO- https://packages.microsoft.com/keys/microsoft.asc | gpg --dearmor > packages.microsoft.gpg
    sudo mv packages.microsoft.gpg /etc/apt/trusted.gpg.d/microsoft.gpg
    sudo sh -c ‘echo “deb [arch=amd64] https://packages.microsoft.com/repos/vscode stable main” > /etc/apt/sources.list.d/vscode.list’
    # Update package index and install VSCode
    sudo apt update
    sudo apt install -y code
    echo “Visual Studio Code installed successfully.”
  else
    echo “Visual Studio Code is already installed.”
  fi
}
# Function to install ruby
install_ruby() {
# Check if ruby command is available
  if ! command -v ruby &>/dev/null; then
    echo “Installing Ruby...”
    sudo apt update
	sudo apt-get install ruby-full build-essential zlib1g-dev
  else
    echo “Ruby is already installed.”
  fi
}
# Function to install MiniConda and Build environment
install_miniconda() {
  if ! command -v conda &>/dev/null; then
    echo “Installing MiniConda...”
    # Download MiniConda installer
    wget -q https://repo.anaconda.com/miniconda/Miniconda3-latest-Linux-x86_64.sh -O ~/miniconda.sh
    # Install MiniConda
    bash ~/miniconda.sh -b -p $HOME/miniconda
    rm ~/miniconda.sh
  else
    echo “MiniConda is already installed.”
  fi
}
# The environment_activate function effectively sets up the required environment configurations by adding lines to the .bashrc file only if they don’t already exist. This ensures that the modifications to the .bashrc file are performed only once and avoids duplication.
environment_activate() {
  add_to_bashrc `# local environment requirement`
  add_to_bashrc ‘export PATH=“$HOME/miniconda/bin:$PATH”’
  add_to_bashrc ‘export GEM_HOME=“$HOME/gems”’
  add_to_bashrc ‘export PATH=“$HOME/gems/bin:$PATH”’
  conda init bash
  echo “Tools and environment setup. Please restart your terminal or run ‘source ~/.bashrc’ to activate it.”
}
# Main script
install_vscode
install_ruby
install_miniconda
environment_activate

```bash
### Sample Docker for KASM workspace ###

FROM kasmweb/core-ubuntu-focal:1.13.1
USER root
ENV HOME /home/kasm-default-profile
ENV STARTUPDIR /dockerstartup
ENV INST_SCRIPTS $STARTUPDIR/install
ENV CONDA_DIR /opt/conda
ENV PATH $CONDA_DIR/bin:$PATH
WORKDIR $HOME
######### Customize Container Here ###########
# Install Google Chrome
COPY ./src/ubuntu/install/chrome $INST_SCRIPTS/chrome/
RUN bash $INST_SCRIPTS/chrome/install_chrome.sh  && rm -rf $INST_SCRIPTS/chrome/
# Install VSCode
COPY ./src/ubuntu/install/vs_code $INST_SCRIPTS/vs_code/
RUN bash $INST_SCRIPTS/vs_code/install_vs_code.sh  && rm -rf $INST_SCRIPTS/vs_code/
# Install Anaconda
RUN cd /tmp/ && wget https://repo.anaconda.com/archive/Anaconda3-2023.07-1-Linux-x86_64.sh \
    && bash Anaconda3-20*-Linux-x86_64.sh -b -p /opt/anaconda3 \
    && rm -r /tmp/Anaconda3-20*-Linux-x86_64.sh \
    && echo ‘source /opt/anaconda3/bin/activate’ >> /etc/bash.bashrc \
    && bash -c “source /opt/anaconda3/bin/activate \
        && conda update -n root conda  \
        && conda update --all \
        && conda clean --all” \
    && /opt/anaconda3/bin/conda config --set ssl_verify /etc/ssl/certs/ca-certificates.crt \
    && /opt/anaconda3/bin/conda install pip \
    && mkdir -p /home/kasm-user/.pip \
    && chown -R 1000:1000 /opt/anaconda3 /home/kasm-default-profile/.conda/ && conda install -y jupyter
######### End Customizations ###########
RUN chown 1000:0 $HOME
RUN $STARTUPDIR/set_user_permission.sh $HOME
ENV HOME /home/kasm-user
WORKDIR $HOME
RUN mkdir -p $HOME && chown -R 1000:0 $HOME
USER 1000 (edi
```

## Automation Tool Setup
Tool installation and verification.  This is a preparation step prior to using Terraform to automate AWS tasks. Below are commands to configure Amazon Web Service Command Line Interface `awscli`, verify configuration `sts`, and test the Simple Storage Service `s3` and `s3api`.

In [ ]:
# 
# Automation key tools
brew install terraform
brew install awscli  

# Credentials setup (~/.aws/credentials)
aws configure  # prompts with keys. user adds values values
# Find User Access Keys in AWS IAM, answer as follows 
<<dialog #begin
AWS Access Key ID [****************PQML]:
AWS Secret Access Key [****************Smcy]:
Default region name [us-west-2]:
Default output format [json]:
dialog #end

# Credentials verification with Security Token Service (sts)
aws sts get-caller-identity
# Notice return of IAM user
<<output #begin
{
  “UserId”: “AIDAX4MZTJQUIUVC2B6D7",
  “Account”: “542024879144",
  “Arn”: “arn:aws:iam::542024879144:user/jm1021"
}
output #begin

# Verify access to view a service, using s3 - 
aws s3 ls
# This is a list of s3 buckets, notice terra-auto directory
<<s3-output #begin
2022-09-28 13:38:43 aws-cloudtrail-logs-542024879144-84383535
2022-08-09 07:00:27 elasticbeanstalk-us-west-1-542024879144
2023-05-25 12:47:32 loginzeen
2022-05-19 01:52:48 prehistoric-pals
2023-07-31 14:14:25 terra-auto
s3-output #ends

# Verify and play with commands to write to a service, using s3api
aws s3api list-objects --bucket terra-auto
aws s3api put-object --bucket terra-auto --key myfile.py
aws s3api list-objects --bucket terra-auto